<a href="https://colab.research.google.com/github/ebahtiyar/Checking-and-Correcting-Grammatical-Errors-in-Turkish-with-Machine-Learning-and-Deep-Learning/blob/main/model_punc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,"/content/drive/My Drive/project")
import os
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sqlite_functions as sql
import functions as f
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import preproccessing as pre
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding,Bidirectional,dot,Activation,concatenate,RepeatVector, TimeDistributed
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import random
from tensorflow import keras
import punc_preprocessing as punc
from keras.callbacks import ModelCheckpoint
import re
#data = pre.load_data("/content/drive/My Drive/project/database.db","Total_senteces_F1")

Mounted at /content/drive


In [ ]:
def control2(data,exc_punc):
    #exc_punc = '"!…'
    s = list()
    se = list()

    for i in data:
        adding = True
        for j in i:
            if j in exc_punc:
                adding = False
        if adding:
            se.append(i)
        
    f1 = '.,?'
    for i in se:
        c = 0
        for j in i:
            if (j in f1):
               c =  c + 1
            
        if c > 1:
           s.append(i)
        
    return s

In [ ]:
def generator_input2(data,exc_punc,balanced):

    
    input_data = list()
    temp = list()
    adding = True
    for line in data:
        adding , s = f.punctuation_filter(line)
        if adding:
           temp.append(s)
           
           
    input_data = control2(temp,exc_punc)
    
    if balanced:
        t_per = punc.n_punc(input_data,":")
        c_pre = punc.n_punc(input_data,";")
        com = punc.n_punc(input_data, ",")
        per = punc.n_punc(input_data, ".")
        que = punc.n_punc(input_data, "?")
        
        total = random.choices(que,k = 250000) + random.choices(per,k = 100000) + random.choices(com,k = 200000)
        total = f.unique(total)
        return total
    else:
        input_data = f.unique(input_data)
        return input_data

In [ ]:
def generator_output2(input_data):
    input = []
    output_data = list()
    for line in input_data:
        output_s = ""
        words = line.split()
        for word in words:
            if word[len(word)-1].isalpha():
               output_s = output_s + "EMP "
            
            else:
               if word[len(word)-1] == ",":
                  output_s = output_s + "COM "
               if word[len(word) -1] == ".":
                  output_s = output_s + "PER "
               if word[len(word) -1]  == "?":
                  output_s = output_s + "QUE "
        s = re.sub(r'[^\w\s]', '', line)
        input.append(s)    
        output_data.append(output_s)
    
    return input,output_data

In [ ]:
data = punc.load_data("/content/drive/My Drive/project/database.db","Total_senteces_F1")
intp = []
for i in data:
    k = i.split()
    if len(k) < 30:
        intp.append(i)
input_data = list()
output_data = list()

In [ ]:
data = generator_input2(intp,exc_punc = '"!…:;',balanced = False )
que = punc.n_punc(intp, "?")
data = data + que 
data = f.unique(data)

In [ ]:
len(que)

152556

In [ ]:
len(data)

817597

In [ ]:
input_data,output_data = generator_output2(data)

In [ ]:
s = {"Training Sentences":input_data,"Labels":output_data}
data_df = pd.DataFrame(s)
data_df.sample(50)
x_train , x_test , y_train , y_test = train_test_split(input_data,output_data,test_size=0.1) 

In [ ]:
data_df.sample(5)

,Training Sentences,Labels
193382,Bu yaşta bir çocuk bunları uyduramaz anlattıkl...,EMP EMP EMP EMP EMP COM EMP PER
695192,Yetenekli hırslı ve çok çalışkandı,COM EMP EMP EMP PER
244755,Dalgalı derin yüzeyin altındaki serin ve sakin...,COM EMP EMP EMP EMP EMP EMP EMP PER
326001,Geleceğe yönelik tüm korkularınızdan uzaklaşma...,EMP EMP EMP EMP COM EMP EMP EMP PER
540001,Peki işi gerçekten de bu noktaya taşımış olabi...,EMP EMP EMP EMP EMP EMP EMP EMP QUE


In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
in_train_tokenized, in_train_tokenizer = tokenize(x_train) 
out_train_tokenized, out_train_tokenizer = tokenize(y_train) 
print('Maximum length input sentence: {}'.format(len(max(in_train_tokenized,key=len))))
print('Maximum length output sentence: {}'.format(len(max(out_train_tokenized,key=len))))

in_test_tokenized, in_test_tokenizer = tokenize(x_test)
out_test_tokenized, out_test_tokenizer = tokenize(y_test)

in_vocab = len(in_train_tokenizer.word_index) + 1
out_vocab = len(out_train_tokenizer.word_index) + 1
print("w vocabulary is of {} unique words".format(in_vocab))
print("r vocabulary is of {} unique words".format(out_vocab))

in_vocab_val = len(in_test_tokenizer.word_index) + 1
out_vocab_val = len(out_test_tokenizer.word_index) + 1

Maximum length input sentence: 29
Maximum length output sentence: 29
w vocabulary is of 410993 unique words
r vocabulary is of 6 unique words


In [ ]:
"""
max_in_len = int(len(max(in_train_tokenized,key=len)))
max_out_len = int(len(max(out_train_tokenized,key=len)))""
"""
max_in_len = 34
max_out_len = 34
in_pad_sentence = pad_sequences(in_train_tokenized, max_in_len, padding = "post")
out_pad_sentence = pad_sequences(out_train_tokenized, max_out_len, padding = "post")

in_pad_sentence = in_pad_sentence.reshape(*in_pad_sentence.shape, 1)
out_pad_sentence = out_pad_sentence.reshape(*out_pad_sentence.shape, 1)

max_in_vlen = int(len(max(in_test_tokenized,key=len)))
max_out_vlen = int(len(max(out_test_tokenized,key=len)))
in_pad_vsentence = pad_sequences(in_test_tokenized, max_in_len, padding = "post")
out_pad_vsentence = pad_sequences(out_test_tokenized, max_out_len, padding = "post")

in_pad_vsentence = in_pad_vsentence.reshape(*in_pad_vsentence.shape, 1)
out_pad_vsentence = out_pad_vsentence.reshape(*out_pad_vsentence.shape, 1)

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

callbacks = [checkpoint]
input_sequence = Input(shape=(max_in_len,))
embedding = Embedding(input_dim=in_vocab, output_dim=128,)(input_sequence)
encoder = Bidirectional(LSTM(64, return_sequences=False))(embedding)
r_vec = RepeatVector(max_out_len)(encoder)
decoder = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3))(r_vec)
logits = TimeDistributed(Dense(out_vocab))(decoder)

model = Model(input_sequence, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_results = model.fit(in_pad_sentence, out_pad_sentence, validation_split = 0.2, epochs=5 , callbacks=callbacks,batch_size = 256,shuffle=True)

Epoch 1/5
2300/2300 [==============================] - ETA: 0s - loss: 0.0780 - accuracy: 0.9708
Epoch 00001: val_loss improved from 0.10049 to 0.09948, saving model to my_best_model.epoch01-loss0.10.hdf5
2300/2300 [==============================] - 567s 246ms/step - loss: 0.0780 - accuracy: 0.9708 - val_loss: 0.0995 - val_accuracy: 0.9659
Epoch 2/5
2300/2300 [==============================] - ETA: 0s - loss: 0.0670 - accuracy: 0.9751
Epoch 00002: val_loss did not improve from 0.09948
2300/2300 [==============================] - 551s 240ms/step - loss: 0.0670 - accuracy: 0.9751 - val_loss: 0.1003 - val_accuracy: 0.9663
Epoch 3/5
2300/2300 [==============================] - ETA: 0s - loss: 0.0580 - accuracy: 0.9784
Epoch 00003: val_loss did not improve from 0.09948
2300/2300 [==============================] - 546s 237ms/step - loss: 0.0580 - accuracy: 0.9784 - val_loss: 0.1026 - val_accuracy: 0.9662
Epoch 4/5
2300/2300 [==============================] - ETA: 0s - loss: 0.0515 - accuracy

In [ ]:
score =model1.evaluate(in_pad_vsentence, out_pad_vsentence)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

2555/2555 [==============================] - 47s 17ms/step - loss: 0.1532 - accuracy: 0.9513
Test loss: 0.15315859019756317 / Test accuracy: 0.9513112306594849


In [ ]:
model1=load_model("/content/my_best_model.epoch01-loss0.10.hdf5")

In [ ]:
sample = "Kim bunlara Çocuk katilleri diye haykıracak"
prediction(model,sample.lower(),in_train_tokenizer,out_train_tokenizer).strip()


'kim bunlara çocuk katilleri diye haykıracak?'

In [ ]:
sample = sample.lower()

In [ ]:
  sample_pad = []
  for word in sample.split():
      if word in in_train_tokenizer.word_index:
         sample_pad.append(in_train_tokenizer.word_index[word])
      else:
         sample_pad.append(1)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')
  pred = model.predict(sample_pad)
  y_id_to_word = {value: key for key, value in out_train_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'
  puncs = [y_id_to_word[np.argmax(x)] for x in pred[0]]

In [ ]:
def prediction(model,sample,in_train_tokenizer,out_train_tokenizer):
  sample_pad = []
  for word in sample.split():
      if word in in_train_tokenizer.word_index:
         sample_pad.append(in_train_tokenizer.word_index[word])
      else:
         sample_pad.append(1)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')
  pred = model.predict(sample_pad)
  y_id_to_word = {value: key for key, value in out_train_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'
  puncs = [y_id_to_word[np.argmax(x)] for x in pred[0]]
  new_sentence = ""
  words = sample.split()
  for n,word in enumerate(words):
     if puncs[n] == "emp":
        new_sentence = new_sentence + word + " "
     if puncs[n] == "com":
        new_sentence = new_sentence + word + ", "
     if puncs[n] == "per":
        new_sentence = new_sentence + word + ".  "    
     if puncs[n] == "que":
        new_sentence = new_sentence + word + "?  "
     if puncs[n] == "<PAD>":
        continue
  return new_sentence

In [ ]:
model1.save('/content/drive/My Drive/project/savedd_model/model_punc_2.h5')

In [ ]:
model = load_model('/content/drive/MyDrive/project/savedd_model/model_punc_1.h5')

In [ ]:
import pickle
with open('/content/drive/MyDrive/project/variables/in_train_tokenizer_punc', 'wb') as handle:
    pickle.dump(in_train_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/project/variables/out_train_tokenizer_punc', 'wb') as handle:
    pickle.dump(out_train_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/My Drive/project/variables/in_train_tokenizer_punc', 'rb') as handle:
    in_train_tokenizer = pickle.load(handle)


with open('/content/drive/My Drive/project/variables/out_train_tokenizer_punc', 'rb') as handle:
    out_train_tokenizer = pickle.load(handle)

**Score**

In [ ]:
data = punc.load_data("/content/drive/My Drive/project/test.db","test_sentences")
intp = []
for i in data:
    k = i.split()
    if len(k) < 30:
        intp.append(i)
input_data = list()
output_data = list()

In [ ]:
data = generator_input2(intp,exc_punc = '"!…:;',balanced = False )
que = punc.n_punc(intp, "?")
data = data[:13000] + que 
data = f.unique(data)

In [ ]:
len(data)

20343

In [ ]:
x_test,y_test = generator_output2(data)

In [ ]:
def prediction_label(model,sample,in_train_tokenizer,out_train_tokenizer):
  sample_pad = []
  for word in sample.split():
      if word in in_train_tokenizer.word_index:
         sample_pad.append(in_train_tokenizer.word_index[word])
      else:
         sample_pad.append(1)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')
  pred = model.predict(sample_pad)
  y_id_to_word = {value: key for key, value in out_train_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'
  puncs = [y_id_to_word[np.argmax(x)] for x in pred[0]]
  return puncs

In [ ]:
test = list(zip(x_test,y_test))
test_x = []
test_y = []
for i,j in test:
    if len(i.split()) == len(j.split()):
       test_x.append(i)
       test_y.append(j)

    

In [ ]:
pred_label = []

for line in test_x:
    pred_label.append(prediction_label(model,line.lower(),in_train_tokenizer,out_train_tokenizer))
test_label = []
for line in test_y:
    line = line.lower()
    test_label.append(line.split())

In [ ]:
pred_l = []

for pred in pred_label:
    t = []
    for i in pred:
        if i != "<PAD>":
           t.append(i)
    pred_l.append(t)

In [ ]:
y = list(zip(test_label,pred_l))
test_input = []
test_label = []
for i,j in y:
    if len(i) == len(j):
       test_input.append(i)
       test_label.append(j)

In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b339eda4be133a8ee2bd35fe42a2736aa1f082f45997e250b3ec3733186a3cb2
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report, f1_score
report = classification_report(test_input ,test_label)
print(report)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: emp seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: com seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: que seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: per seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

          er       0.96      0.98      0.97     12426
          mp       0.42      0.29      0.34     36083
          om       0.52      0.17      0.25     19041
          ue       0.97      0.93      0.95      6875

   micro avg       0.64      0.43      0.52     74425
   macro avg       0.72      0.59      0.63     74425
weighted avg       0.59      0.43      0.48     74425

